# Evaluation of the difference between ordinary mlp and spark mlp

## 1. Import library

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

## 2. Load data

In [ ]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

## 3. Prepocessing

In [3]:
train_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
train_df['Sex'] = train_df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
train_df['Embarked'] = train_df['Embarked'].apply(lambda x: 0 if x == 'S' else 1)

In [5]:
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## 4. Modelling

In [7]:
def build_and_train_model(max_iter, layer_sizes, block_size, tol, step_size, solver):
    model = Sequential()
    model.add(Dense(layer_sizes[0], input_dim=X_train.shape[1], activation='relu'))
    
    for size in layer_sizes[1:]:
        model.add(Dense(size, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=step_size),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=max_iter, batch_size=block_size, verbose=0)
    
    return model

## 5. Retrive the time needed to tune one set of hyperparameters

In [8]:
maxIters = [10, 100, 500, 1000, 1500, 1800]
layers = [[7, 4, 2], [7, 8, 4, 2], [7, 32, 64, 16, 2]]
blockSizes = [16, 64, 128, 256]
tol = 0.000001
step_size = 0.03
solver = 'l-bfgs'
model = build_and_train_model(maxIters[0], layers[0], blockSizes[0], tol, step_size, solver)

for max_iter in maxIters:
    results = []
    for layer in layers:
        for block_size in blockSizes:
            model = build_and_train_model(max_iter, layer, block_size, tol, step_size, solver)
            y_pred = model.predict(X_test).flatten()
            y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]
            np.savetxt(f'result/og-mlp/predictions_{max_iter}_{layer[1]}_{block_size}.csv', y_pred, delimiter=',')
            results.append({'layers': layer, 'block_size': block_size, 'acc': accuracy_score(y_test, y_pred)})
    results_df = pd.DataFrame(results)
    results_df.to_csv(f'result/og-mlp/results_{max_iter}.csv', index=False)

/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
